In [30]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score, make_scorer
from common import get_train, get_test, get_parking, feat_eng, premodel_formating, mean_target_enc, get_XY
import pandas as pd
from sklearn.cross_validation import cross_val_score
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
%matplotlib inline
from sklearn.model_selection import KFold

In [31]:
train_df = get_train()
test_df = get_test()

train_df = feat_eng(train_df)
test_df = feat_eng(test_df)

Time it took: 0.14241504669189453
Time it took: 0.09323358535766602
adding additional features to test and/or train ...

loading parking data 1.7M
loading address data 30K
calculating closest joins
merging
complete
fixing neighborhood string formatting
merging in neighborhood information
Time it took: 0.3869357109069824
adding additional features to test and/or train ...

loading parking data 1.7M
loading address data 30K
calculating closest joins
merging
complete
fixing neighborhood string formatting
merging in neighborhood information
Time it took: 0.3875710964202881


In [14]:
# tr, vl = premodel_formating(train_df)
# y_tr = tr['any_spot'].values
# y_vl = vl['any_spot'].values
# X_tr = tr[[col for col in tr.columns if col not in ['any_spot','Real.Spots']]].copy()
# X_vl = vl[[col for col in tr.columns if col not in ['any_spot','Real.Spots']]].copy()
# print(y_tr.shape, X_tr.shape, y_vl.shape, X_vl.shape)

(770,) (770, 18) (330,) (330, 18)


## Random Forest ##

In [32]:
train_df = premodel_formating(train_df, split=False)

In [33]:
X = train_df.drop(["Real.Spots", "any_spot"], axis=1)
y = train_df["any_spot"]

In [34]:
def cv_k(X, y, k, model):
    kf = KFold(n_splits = k)
    kf.get_n_splits(X)
    # f0.5 score
    f_score = 0
    # precision
    p_score = 0
    # recall
    r_score = 0
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.ix[train_index], X.ix[val_index]    
        y_train, y_val = y.ix[train_index], y.ix[val_index]
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        f_score += fbeta_score(y_val, preds, beta=0.5)
        p_score += precision_score(y_val, preds)
        r_score += recall_score(y_val, preds)
    return [f_score/(k*1.0), p_score/(k*1.0), r_score/(k*1.0)]

### Parameter Tuning

Model1: n_estimators=16,min_samples_leaf=5,max_features=0.9, max_depth=5

In [10]:
import warnings; warnings.simplefilter('ignore')

In [11]:
# Tune three parameters: n_estimators, min_samepls_leaf, max_features and max_depth
numOfestimators = np.arange(18) + 1
numOfleafs = np.arange(15) + 1
numOffeatures = np.arange(0.1, 1.1, 0.2)
depths = np.arange(0, 7) + 1
best_result = []
for numOfestimator in numOfestimators:
    for numOfleaf in numOfleafs:
        for numOffeature in numOffeatures:  
            for depth in depths:
                result = [numOfestimator, numOfleaf, numOffeature, depth]
                m = RandomForestClassifier(n_jobs=-1, n_estimators=numOfestimator,\
                                    min_samples_leaf=numOfleaf,\
                                    max_features=numOffeature, max_depth = depth)
                score = cv_k(X, y, 3, m)
                result.append(score[0])
                result.append(score[1])
                if len(best_result) == 0: best_result = result
                elif best_result[5] < result[5]: 
                    print(result)
                    best_result = result

[1, 1, 0.10000000000000001, 2, 0.27149084568439408, 0.12387387387387387]
[1, 1, 0.10000000000000001, 3, 0.32419441406686489, 0.25927748413207419]
[1, 1, 0.10000000000000001, 4, 0.52261076302863974, 0.3742517046277965]
[1, 1, 0.10000000000000001, 7, 0.61086273561045823, 0.43284393423910411]
[1, 1, 0.30000000000000004, 6, 0.65389562396214285, 0.44080495300007488]
[1, 1, 0.30000000000000004, 7, 0.63895907654441741, 0.47322742493394099]
[1, 1, 0.70000000000000007, 5, 0.63164475117104446, 0.49022530596258634]
[1, 1, 0.90000000000000013, 5, 0.61870360596812823, 0.514423454115715]
[1, 5, 0.70000000000000007, 5, 0.60975509566251807, 0.53570588679165698]
[1, 10, 0.30000000000000004, 5, 0.61394678076784848, 0.53801541192608726]
[1, 11, 0.90000000000000013, 6, 0.6283385945029113, 0.56998286051419556]
[4, 11, 0.30000000000000004, 7, 0.68731190609968384, 0.5924782514255732]


### RF without mean encoding

In [29]:
# 4, 11, 0.30000000000000004, 7,
# leaderboard score 0.57071
m = RandomForestClassifier(n_jobs=-1, n_estimators=4,min_samples_leaf=11,max_features=0.3, max_depth = 7)
m.fit(X, y)
cv_k(X, y, 3, m)

[0.51251245806134982, 0.6303347937810807, 0.3578299842809099]

### Prep for submission

In [3]:
trn = premodel_formating(train_df, split=False)
tst = premodel_formating(test_df, split=False, test=True)

In [4]:
trn, tst = mean_target_enc(trn, tst)
print(trn.shape, tst.shape)

validation or test nulls 75, will fill with 0.4
(1100, 24) (726, 22)


In [6]:
trn.shape, tst.shape

((1100, 24), (726, 22))

In [7]:
X_tr, y_tr = get_XY(trn)

In [8]:
X_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 1099
Data columns (total 22 columns):
Street.Length       1100 non-null float64
holiday             1100 non-null int64
dow                 1100 non-null int64
mon                 1100 non-null int64
hour                1100 non-null int64
morn_night          1100 non-null int64
From_lat            1100 non-null float64
From_long           1100 non-null float64
From_zip            1100 non-null int64
To_lat              1100 non-null float64
To_long             1100 non-null float64
To_zip              1100 non-null int64
nhood               1100 non-null int64
zipcode             1100 non-null int64
area                1100 non-null float64
f_pop               1100 non-null float64
m_pop               1100 non-null float64
pop                 1100 non-null float64
mean_spot_prob      1100 non-null float64
mean_spot_ct        1100 non-null float64
z_mean_spot_prob    1100 non-null float64
z_mean_spot_ct      1100 non-

In [9]:
print(X_tr.shape, y_tr.shape)

(1100, 22) (1100,)


In [24]:
rfc = RandomForestClassifier(n_estimators= 10, max_depth=10)
m = rfc.fit(X_tr, y_tr)

In [25]:
xgc = XGBClassifier(max_depth = 5, n_estimators=100)
m = xgc.fit(X_tr, y_tr)

In [26]:
tst_pred = m.predict(tst)
to_sub = pd.DataFrame({'id': range(1, len(tst_pred)+1), 'any_spot': tst_pred})

In [27]:
to_sub[['id','any_spot']].to_csv('submissions/mean_enc_sub.csv', index=False)
to_sub.shape

(726, 2)